<a href="https://colab.research.google.com/github/CodeCreator333/colab-projects/blob/main/screener_strategie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from dataclasses import dataclass
from datetime import datetime, timedelta

def get_stock_data(ticker, start_date, end_date):
    stock = yf.Ticker(ticker)
    hist = stock.history(start=start_date, end=end_date)
    df = pd.DataFrame({
        "Date": hist.index,
        "Close": hist['Close'].values,
        "High": hist['High'].values,
        "Low": hist['Low'].values,
        "Open": hist['Open'].values,
        "Volume": hist['Volume'].values
    })
    df.set_index("Date", inplace=True)
    return df

@dataclass
class BacktestResult:
    ticker: str
    breakout_signals: int
    breakout_successes: int
    ma_rsi_signals: int
    ma_rsi_successes: int

def calculate_indicators(df):
    df['MA20'] = df['Close'].rolling(window=20).mean()
    df['MA50'] = df['Close'].rolling(window=50).mean()
    df['MA200'] = df['Close'].rolling(window=200).mean()

    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))

    df['ATR'] = df[['High', 'Low', 'Close']].apply(
        lambda x: max(x['High'] - x['Low'],
                     abs(x['High'] - x['Close']),
                     abs(x['Low'] - x['Close'])), axis=1
    ).rolling(window=14).mean()

    df['Volume_MA'] = df['Volume'].rolling(window=20).mean()

    return df

def optimized_breakout_strategy(df):
    signals = []
    successes = 0

    for i in range(201, len(df) - 5):
        price = df['Close'].iloc[i]
        high_20 = df['High'].iloc[i-20:i].max()
        volume_ratio = df['Volume'].iloc[i] / df['Volume_MA'].iloc[i] if df['Volume_MA'].iloc[i] > 0 else 0
        rsi = df['RSI'].iloc[i]
        ma20 = df['MA20'].iloc[i]
        ma50 = df['MA50'].iloc[i]
        atr = df['ATR'].iloc[i]

        if (price > high_20 and
            volume_ratio > 1.3 and
            35 < rsi < 75 and
            ma20 > ma50):

            signals.append(i)
            entry_price = price
            target_price = entry_price * 1.03
            stop_loss = entry_price - (atr * 2)

            for j in range(i+1, min(i+6, len(df))):
                future_price = df['Close'].iloc[j]
                if future_price >= target_price:
                    successes += 1
                    break
                elif future_price <= stop_loss:
                    break

    return len(signals), successes

def optimized_ma_rsi_strategy(df):
    signals = []
    successes = 0

    for i in range(201, len(df) - 5):
        price = df['Close'].iloc[i]
        ma20 = df['MA20'].iloc[i]
        ma50 = df['MA50'].iloc[i]
        ma200 = df['MA200'].iloc[i]
        rsi = df['RSI'].iloc[i]
        prev_rsi = df['RSI'].iloc[i-1]
        volume_ratio = df['Volume'].iloc[i] / df['Volume_MA'].iloc[i] if df['Volume_MA'].iloc[i] > 0 else 0
        atr = df['ATR'].iloc[i]

        if (rsi < 40 and
            prev_rsi < rsi and
            price > ma200 and
            volume_ratio > 1.0):

            signals.append(i)
            entry_price = price
            target_price = entry_price * 1.025
            stop_loss = entry_price - (atr * 2)

            for j in range(i+1, min(i+6, len(df))):
                future_price = df['Close'].iloc[j]
                if future_price >= target_price:
                    successes += 1
                    break
                elif future_price <= stop_loss:
                    break

    return len(signals), successes

stock_list = [
"ADBE", "AMD", "ABNB", "GOOGL", "GOOG", "AMZN", "AEP", "AMGN", "ADI", "AAPL",
"AMAT", "APP", "ARM", "ASML", "AZN", "TEAM", "ADSK", "ADP", "AXON", "BKR",
"BIIB", "BKNG", "AVGO", "CDNS", "CDW", "CHTR", "CTAS", "CSCO", "CCEP", "CTSH",
"CMCSA", "CEG", "CPRT", "CSGP", "COST", "CRWD", "CSX", "DDOG", "DXCM", "FANG",
"DASH", "EA", "EXC", "FAST", "FTNT", "GEHC", "GILD", "GFS", "HON", "IDXX",
"INTC", "INTU", "ISRG", "KDP", "KLAC", "KHC", "LRCX", "LIN", "LULU", "MAR",
"MRVL", "MELI", "META", "MCHP", "MU", "MSFT", "MSTR", "MDLZ", "MNST", "NFLX",
"NVDA", "NXPI", "ORLY", "ODFL", "ON", "PCAR", "PLTR", "PANW", "PAYX", "PYPL",
"PDD", "PEP", "QCOM", "REGN", "ROPU", "ROST", "SHOP", "SBUX", "SNPS", "TMUS",
"TTWO", "TSLA", "TXN", "TRI", "TTD", "VRSK", "VRTX", "WBD", "WDAY", "XEL", "ZS"
]

end_date = datetime.now()
start_date = end_date - timedelta(days=730)

results = []

for ticker in stock_list:
    df = get_stock_data(ticker, start_date, end_date)
    df = calculate_indicators(df)

    breakout_signals, breakout_successes = optimized_breakout_strategy(df)
    ma_rsi_signals, ma_rsi_successes = optimized_ma_rsi_strategy(df)

    result = BacktestResult(
        ticker=ticker,
        breakout_signals=breakout_signals,
        breakout_successes=breakout_successes,
        ma_rsi_signals=ma_rsi_signals,
        ma_rsi_successes=ma_rsi_successes
    )
    results.append(result)

print(f"{'Ticker':<6} | {'Breakout Signals':>16} | {'Breakout Successes':>18} | {'Success Rate':>12} | {'MA/RSI Signals':>14} | {'MA/RSI Successes':>16} | {'Success Rate':>12}")
print("-" * 110)

for result in results:
    breakout_rate = (result.breakout_successes / result.breakout_signals * 100) if result.breakout_signals > 0 else 0
    ma_rsi_rate = (result.ma_rsi_successes / result.ma_rsi_signals * 100) if result.ma_rsi_signals > 0 else 0

    print(f"{result.ticker:<6} | {result.breakout_signals:>16} | {result.breakout_successes:>18} | {breakout_rate:>11.2f}% | {result.ma_rsi_signals:>14} | {result.ma_rsi_successes:>16} | {ma_rsi_rate:>11.2f}%")

ERROR:yfinance:$ROPU: possibly delisted; no timezone found


Ticker | Breakout Signals | Breakout Successes | Success Rate | MA/RSI Signals | MA/RSI Successes | Success Rate
--------------------------------------------------------------------------------------------------------------
ADBE   |                0 |                  0 |        0.00% |              0 |                0 |        0.00%
AMD    |                4 |                  2 |       50.00% |              0 |                0 |        0.00%
ABNB   |                3 |                  1 |       33.33% |              0 |                0 |        0.00%
GOOGL  |                6 |                  2 |       33.33% |              3 |                3 |      100.00%
GOOG   |                5 |                  2 |       40.00% |              2 |                2 |      100.00%
AMZN   |                7 |                  2 |       28.57% |              5 |                1 |       20.00%
AEP    |                2 |                  0 |        0.00% |              5 |                0 